In [2]:
import sys
from pathlib import Path

# Add project root to sys.path (so config.py is found)
root = Path().resolve().parent  # one level up from notebooks/
sys.path.append(str(root))

from config import RAW_DIR, INTERIM_DIR
import pandas as pd

print("RAW_DIR:", RAW_DIR)
print("INTERIM_DIR:", INTERIM_DIR)


RAW_DIR: C:\DataProjects\uh-ds-housing-data\data\raw
INTERIM_DIR: C:\DataProjects\uh-ds-housing-data\data\interim


# Step 3 — Prepare Processed Dataset

**Goal:**  
Convert the interim dataset into a clean, modeling-ready version stored in `PROCESSED_DIR`.

We'll:
1. Load the interim dataset  
2. Handle missing values & datatypes  
3. Engineer features (year, month, log price, high-value flag)  
4. Save final processed dataset for modeling


In [26]:
# === 03_prepare_processed.ipynb ===
import pandas as pd, numpy as np
from config import INTERIM_DIR, PROCESSED_DIR

# -----------------------------
# 1) Load exact interim file
# -----------------------------
interim_path = INTERIM_DIR / "uk_housing_2010_2017.csv"
if not interim_path.exists():
    raise FileNotFoundError(f"Expected file not found: {interim_path}")

print(f" Loading interim dataset from: {interim_path}")
# Read as strings to control parsing ourselves (dates/prices vary)
df = pd.read_csv(interim_path, low_memory=False, dtype=str)
print(f" Loaded {len(df):,} rows and {df.shape[1]} columns")

# -----------------------------
# 2) Rename columns canonically
# -----------------------------
rename_map = {
    "Transaction unique identifier": "transaction_id",
    "Price": "price",
    "Date of Transfer": "date_of_transfer",
    "Property Type": "property_type",
    "Old/New": "new_build_flag",
    "Duration": "tenure_type",
    "Town/City": "town_city",
    "District": "local_authority",
    "County": "county",
    "PPDCategory Type": "ppd_category",
    "Record Status - monthly file only": "record_status",
}
# only rename those that exist
df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns}, inplace=True)

# ensure required core columns exist
required = ["price","date_of_transfer"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f"Missing required columns after rename: {missing}. Present: {df.columns.tolist()}")

# -----------------------------
# 3) Clean & coerce 'price'
# -----------------------------
# strip currency symbols/commas/spaces, then to numeric
df["price"] = (
    df["price"]
      .astype(str)
      .str.replace(r"[^\d.]", "", regex=True)  # remove £, commas, spaces, etc.
      .replace({"": np.nan})
)
df["price"] = pd.to_numeric(df["price"], errors="coerce")

# -----------------------------
# 4) Parse dates robustly
# -----------------------------
def parse_dates(s: pd.Series) -> pd.Series:
    # Try several explicit patterns and pick the first that yields majority non-NaT
    candidates = [
        ("%d/%m/%Y", {"dayfirst": True}),
        ("%Y-%m-%d", {}),
        ("%d/%m/%Y %H:%M", {"dayfirst": True}),
        ("%Y-%m-%d %H:%M:%S", {}),
        ("%d-%m-%Y", {"dayfirst": True}),
        ("%d-%b-%Y", {"dayfirst": True}),  # e.g., 01-Jan-2017
    ]
    best = None; best_valid = -1
    for fmt, kw in candidates:
        dt = pd.to_datetime(s, format=fmt, errors="coerce", **kw)
        valid = dt.notna().sum()
        if valid > best_valid:
            best, best_valid = dt, valid
        if valid > 0.7 * len(s):
            break
    if best is None or best.notna().sum() == 0:
        # final fallback: dateutil with dayfirst
        best = pd.to_datetime(s, errors="coerce", dayfirst=True)
    return best

df["date_of_transfer"] = parse_dates(df["date_of_transfer"])

# -----------------------------
# 5) Drop invalid rows & dupes
# -----------------------------
before = len(df)
df = df[df["price"].notna() & (df["price"] > 0) & df["date_of_transfer"].notna()].copy()

if "transaction_id" in df.columns:
    df.drop_duplicates(subset=["transaction_id"], inplace=True)
else:
    # safe fallback de-duplication
    subset_cols = [c for c in ["date_of_transfer","price","postcode","property_type","town_city","local_authority","county"] if c in df.columns]
    df.drop_duplicates(subset=subset_cols if subset_cols else None, inplace=True)

print(f" After cleaning: {len(df):,} rows (dropped {before - len(df):,})")
if len(df) > 0:
    print(f" Date range: {df['date_of_transfer'].min().date()} → {df['date_of_transfer'].max().date()}")

# -----------------------------
# 6) Filter to 2010–2017
# -----------------------------
if len(df) > 0:
    start, end = pd.Timestamp("2010-01-01"), pd.Timestamp("2017-12-31")
    mask = (df["date_of_transfer"] >= start) & (df["date_of_transfer"] <= end)
    kept = mask.sum()
    df = df.loc[mask].copy()
    print(f" Kept {kept:,} rows in 2010–2017 window")

# -----------------------------
# 7) Feature engineering
# -----------------------------
if len(df) > 0:
    df["year"] = df["date_of_transfer"].dt.year
    df["month"] = df["date_of_transfer"].dt.month
    df["log_price"] = np.log1p(df["price"])

    # derive high-value = top 20% within local_authority-year (fallback to year if LA missing)
    group_cols = ["local_authority","year"] if "local_authority" in df.columns else ["year"]
    df["price_80th"] = df.groupby(group_cols)["price"].transform(lambda s: s.quantile(0.80))
    df["is_high_value"] = (df["price"] >= df["price_80th"]).astype("Int8")
    df.drop(columns=["price_80th"], inplace=True)

    hv_share = float(df["is_high_value"].mean()) if df["is_high_value"].notna().any() else np.nan
    print(" High-value share:", None if np.isnan(hv_share) else round(hv_share, 3))
else:
    # create empty columns to keep schema stable
    for c in ["year","month","log_price","is_high_value"]:
        if c not in df.columns:
            df[c] = pd.Series(dtype="float64" if c!="is_high_value" else "Int8")

# -----------------------------
# 8) Missing summary (top 10)
# -----------------------------
if len(df) > 0:
    na_counts = df.isna().sum()
    na_pct = (na_counts / len(df) * 100).round(2)
    miss = pd.DataFrame({"Missing Count": na_counts, "Missing %": na_pct})
    print("\n Missing Value Summary (top 10):")
    print(miss[miss["Missing %"] > 0].sort_values("Missing %", ascending=False).head(10))
else:
    print("\n No rows after cleaning. Check date parsing and price coercion above.")

# -----------------------------
# 9) Save processed CSV
# -----------------------------
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR / "uk_housing_2010_2017_processed.csv"
df.to_csv(out_path, index=False)
print(f"\n Saved processed dataset to: {out_path}")
print(f" Final shape: {df.shape}")


 Loading interim dataset from: C:\DataProjects\uh-ds-housing-data\data\interim\uk_housing_2010_2017.csv
 Loaded 6,200,823 rows and 10 columns
 After cleaning: 5,944,841 rows (dropped 255,982)
 Date range: 2010-01-01 → 2017-06-29
 Kept 5,944,841 rows in 2010–2017 window
 High-value share: 0.201

 Missing Value Summary (top 10):
Empty DataFrame
Columns: [Missing Count, Missing %]
Index: []

 Saved processed dataset to: C:\DataProjects\uh-ds-housing-data\data\processed\uk_housing_2010_2017_processed.csv
 Final shape: (5944841, 14)
